In [69]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


# loading labelled training data
train = pd.read_csv( "labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

# loading labelled testing data
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )

# loading unlabelled training data
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

print "Read %d labeled train reviews, %d labeled test reviews, and %d unlabeled reviews\n" % (train["review"].size,  test["review"].size, unlabeled_train["review"].size )

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [70]:
def review_to_wordlist( review, remove_stopwords=False ):
    words = review.lower().split()

    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    # 5. Return a list of words
    return(words)

In [71]:
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    review_text = BeautifulSoup(review).get_text()
    
    review_text_1 = re.sub("[^a-zA-Z]"," ", review_text)
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review_text_1)

    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, remove_stopwords ))
    
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [72]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set
Parsing sentences from unlabeled set


In [75]:
print len(sentences)

74995


In [76]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2017-08-27 14:36:27,846 : INFO : 'pattern' package not found; tag filters are not available for English
2017-08-27 14:36:27,850 : INFO : collecting all words and their counts
2017-08-27 14:36:27,851 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2017-08-27 14:36:28,757 : INFO : PROGRESS: at sentence #10000, processed 2385328 words, keeping 51525 word types
2017-08-27 14:36:29,669 : INFO : PROGRESS: at sentence #20000, processed 4747257 words, keeping 67811 word types
2017-08-27 14:36:30,534 : INFO : PROGRESS: at sentence #30000, processed 7099915 words, keeping 81670 word types
2017-08-27 14:36:31,499 : INFO : PROGRESS: at sentence #40000, processed 9467635 words, keeping 93389 word types
2017-08-27 14:36:32,313 : INFO : PROGRESS: at sentence #50000, processed 11865624 words, keeping 103474 word types
2017-08-27 14:36:33,250 : INFO : PROGRESS: at sentence #60000, processed 14248671 words, keeping 112172 word types
2017-08-27 14:36:33,884 : INFO : PROGRESS: at sentence #70000, processed 16609786 words, keeping 119828 word types
2017-08-27 14:36:34,366 : INFO : collected 123500 word types from a corpus of 17796947 raw words and 74995 sentences
2017-08-27 14:36:34,368 : INFO : Loading a fresh vocabulary
2017-08-27 14:36:35,096 : 

2017-08-27 14:37:40,169 : INFO : PROGRESS: at 45.39% examples, 447510 words/s, in_qsize 8, out_qsize 0
2017-08-27 14:37:41,182 : INFO : PROGRESS: at 46.00% examples, 446618 words/s, in_qsize 8, out_qsize 0
2017-08-27 14:37:42,193 : INFO : PROGRESS: at 46.89% examples, 448189 words/s, in_qsize 8, out_qsize 0
2017-08-27 14:37:43,204 : INFO : PROGRESS: at 47.54% examples, 447612 words/s, in_qsize 8, out_qsize 0
2017-08-27 14:37:44,226 : INFO : PROGRESS: at 48.43% examples, 449037 words/s, in_qsize 8, out_qsize 0
2017-08-27 14:37:45,230 : INFO : PROGRESS: at 49.05% examples, 448239 words/s, in_qsize 8, out_qsize 0
2017-08-27 14:37:46,241 : INFO : PROGRESS: at 49.90% examples, 449475 words/s, in_qsize 8, out_qsize 0
2017-08-27 14:37:47,252 : INFO : PROGRESS: at 50.50% examples, 448528 words/s, in_qsize 7, out_qsize 2
2017-08-27 14:37:48,254 : INFO : PROGRESS: at 51.41% examples, 450392 words/s, in_qsize 8, out_qsize 0
2017-08-27 14:37:49,265 : INFO : PROGRESS: at 52.04% examples, 449763 wor

2017-08-27 14:38:53,407 : INFO : saved 300features_40minwords_10context


In [94]:
model.doesnt_match("france england germany berlin".split())

'berlin'

In [95]:
model.doesnt_match("paris berlin london austria".split())

'london'

In [97]:
model.most_similar("awful")

[(u'terrible', 0.8170218467712402),
 (u'horrible', 0.7606260776519775),
 (u'atrocious', 0.7489117383956909),
 (u'dreadful', 0.7323407530784607),
 (u'horrendous', 0.7301393151283264),
 (u'abysmal', 0.7289620637893677),
 (u'horrid', 0.6970181465148926),
 (u'lousy', 0.6664386987686157),
 (u'appalling', 0.6595410108566284),
 (u'laughable', 0.6260790824890137)]

In [99]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

2017-08-27 15:20:32,172 : INFO : loading Word2Vec object from 300features_40minwords_10context
2017-08-27 15:20:32,287 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2017-08-27 15:20:32,288 : INFO : setting ignored attribute syn0norm to None
2017-08-27 15:20:32,289 : INFO : setting ignored attribute cum_table to None
2017-08-27 15:20:32,290 : INFO : loaded 300features_40minwords_10context


In [102]:
# lets clean the review

def clean_reviews(review,remove_stopwords):
    clean_1 = BeautifulSoup(review).get_text()
    clean_2 = re.sub("[^a-zA-Z]"," ",clean_1)
    clean_3 = clean_2.lower().split()
    if remove_stopwords:
        stop_words = set(stopwords.words("english"))
        words = [w for w in clean_3 if not w in stop_words]
    
    return words
    
clean_train_reviews = []

for review in train["review"]:
    clean_train_reviews.append( clean_reviews( review, remove_stopwords=True ))
    


In [118]:
# model.wv.syn0norm instead of  model.syn0norm
# model.wv.syn0 instead of  model.syn0
# model.wv.vocab instead of model.vocab
# model.wv.index2word instead of  model.index2word
import numpy as np

def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    
    for word in words:
        # if the word is in wordset then add to feature vec
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    
    featureVec = np.divide(featureVec,nwords)
    return featureVec

def get_average_feature_vectors( reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    
    for review in reviews:
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter += 1
        return reviewFeatureVecs



In [119]:
trainDataVecs = get_average_feature_vectors( clean_train_reviews, model, num_features )

In [120]:
print "Creating average feature vecs for test reviews"
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( clean_reviews( review, remove_stopwords=True ))

testDataVecs = get_average_feature_vectors( clean_test_reviews, model, num_features )

Creating average feature vecs for test reviews


In [123]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print "Fitting a random forest to labeled training data..."
forest = forest.fit( trainDataVecs, train["sentiment"] )
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )


Fitting a random forest to labeled training data...


In [132]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] / 5

kmeans_clustering = KMeans( n_clusters = num_clusters )

idx = kmeans_clustering.fit_predict( word_vectors )

end = time.time()

elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

Time taken for K Means clustering:  757.010732174 seconds.


In [154]:
word_centroid_map = dict(zip( model.wv.index2word, idx ))

print len(set(word_centroid_map.values()))


3297


In [146]:
for cluster in xrange(0,10):
    print "\nCluster %d" % cluster
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
[u'diversion', u'romp']

Cluster 1
[u'lesley', u'nailed', u'burns']

Cluster 2
[u'bolts', u'fire', u'carrying', u'fires', u'blasting', u'tilt', u'toting', u'shooting', u'firing']

Cluster 3
[u'deliberate', u'deliberately', u'purposely']

Cluster 4
[u'messiah', u'ruler', u'antichrist', u'protector', u'prophecy', u'emperor']

Cluster 5
[u'object', u'implication', u'orgasm', u'ego', u'symbol']

Cluster 6
[u'altering', u'examining', u'reflecting']

Cluster 7
[u'freed', u'governor', u'assassinated', u'president', u'senator', u'minister', u'probation']

Cluster 8
[u'bacon', u'costner', u'smith', u'kevin', u'spacey', u'kline']

Cluster 9
[u'trite', u'humorless', u'unremarkable', u'aimless', u'uninspiring', u'vapid', u'overwrought', u'stale', u'lifeless', u'insipid']


In [155]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids


In [156]:
train_centroids = np.zeros( (train["review"].size, num_clusters), dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, word_centroid_map )
    counter += 1

In [157]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print "Fitting a random forest to labeled training data..."
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})

Fitting a random forest to labeled training data...


In [166]:
output['sentiment'][1]

0

In [167]:
test['review'][1]

'"This movie is a disaster within a disaster film. It is full of great action scenes, which are only meaningful if you throw away all sense of reality. Let\'s see, word to the wise, lava burns you; steam burns you. You can\'t stand next to lava. Diverting a minor lava flow is difficult, let alone a significant one. Scares me to think that some might actually believe what they saw in this movie.<br /><br />Even worse is the significant amount of talent that went into making this film. I mean the acting is actually very good. The effects are above average. Hard to believe somebody read the scripts for this and allowed all this talent to be wasted. I guess my suggestion would be that if this movie is about to start on TV ... look away! It is like a train wreck: it is so awful that once you know what is coming, you just have to watch. Look away and spend your time on more meaningful content."'